In [1]:
%run ../template.ipynb

In [2]:
exp_id = "050318"
series_names = ["mainstream", "maxsharing", "nosharing"]
num_apps_range = range(3, 30+1)
# num_apps_range = range(3, 5+1)

In [3]:
setups = {}
for num_apps in num_apps_range:
    setups.update(dataloaders.load_setups(exp_id,
                                          setup_file_str="/setups.{exp_id}-" + str(num_apps) + "{version}.pickle",
                                          legacy='InconsistentIds'))

Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentIds... Done (100 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentIds... Done (0 loaded)
Loading setups... Using InconsistentId

In [4]:
rows = []
for series_name in series_names:
    schedules = []
    for num_apps in num_apps_range:
        schedules += dataloaders.load_schedules("050318", "greedy." + series_name + ".sim.*-" + str(num_apps) + ".v1", variant='v1+metrics', setups=setups)
    # Extract some attributes from the schedules.
    # Add on some based on the file name.
    rows += ex(schedules,
               each=lambda s: s.to_map({'correlation': mean([app['correlation_coefficient'] for app in s._apps])}),
               constant={'scheduler': 'greedy', 'sharing': series_name})
df = comb(rows)

In [5]:
df.groupby('num_apps').apply(len)

num_apps
3     1800
4     1800
5     1800
6     1800
7     1800
8     1800
9     1800
10    1800
15    1800
20    1800
25    1800
30      52
dtype: int64

In [6]:
import seaborn as sns
df['corr_bin'] = df['correlation'].map(lambda x: round(x * 5) / 5)
df_view = df[df['sharing'] == 'mainstream']
df_view.groupby(['num_apps', 'corr_bin'])['f1'].mean()
# for i in num_apps_range:
#     df_view = df[df['num_apps'] == i]
# #     plt.plot(df_view.groupby('correlation')['f1'].mean())
#     print i
#     sns.jointplot(df_view['correlation'], df_view['f1'], kind="kde")


num_apps  corr_bin
3         0.2         0.742013
          0.4         0.722835
          0.6         0.718458
          0.8         0.687967
4         0.2         0.663228
          0.4         0.731357
          0.6         0.696788
          0.8         0.746135
5         0.2         0.726465
          0.4         0.705118
          0.6         0.708765
          0.8         0.734243
6         0.2         0.690255
          0.4         0.701280
          0.6         0.705586
          0.8         0.686487
7         0.2         0.763449
          0.4         0.708869
          0.6         0.694520
          0.8         0.723236
8         0.2         0.753909
          0.4         0.692887
          0.6         0.704916
9         0.2         0.728025
          0.4         0.696051
          0.6         0.701810
10        0.2         0.721363
          0.4         0.692274
          0.6         0.705409
15        0.4         0.690342
          0.6         0.706217
20        0.4       

In [7]:
# See Pandas: Group By: split-apply-combine
# https://pandas.pydata.org/pandas-docs/stable/groupby.html
df_view = df[df['budget'] == 100]
# Group <setups> by number of apps, aggregate by mean.
# Make sure that the right-most index is the x-axis.
grouped = df_view.groupby(['sharing', 'num_apps'])

# Original way (and most flexible) of doing it
xss, yss = agg2xy(grouped['f1'].mean(), names=series_names)
series = get_series(xss, yss,
                    names=series_names,
                    plotparams=dict(lw=2, markersize=8))

# Neater way to do it if you don't need xss/yss
series2 = agg2series(grouped['fps'].mean(),
                     names=series_names,
                     plotstyles=styles.SERIES_ALT,
                     plotparams=dict(lw=1, markersize=8, alpha=0.7, linestyle='--'))